In [10]:
import pytraj as pt, nglview as nv
import numpy as np

In [11]:
!wget http://ambermd.org/tutorials/advanced/tutorial16/include/DOPC_128.inpcrd
!wget http://ambermd.org/tutorials/advanced/tutorial16/include/DOPC_128.prmtop
traj = pt.load('DOPC_128.inpcrd', 'DOPC_128.prmtop')

--2016-06-13 13:32:51--  http://ambermd.org/tutorials/advanced/tutorial16/include/DOPC_128.inpcrd
Resolving ambermd.org... 165.230.17.132
Connecting to ambermd.org|165.230.17.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1163786 (1.1M) [text/plain]
Saving to: 'DOPC_128.inpcrd.1'

DOPC_128.inpcrd.1   100%[===================>]   1.11M  --.-KB/s    in 0.08s   

2016-06-13 13:32:52 (14.8 MB/s) - 'DOPC_128.inpcrd.1' saved [1163786/1163786]

--2016-06-13 13:32:52--  http://ambermd.org/tutorials/advanced/tutorial16/include/DOPC_128.prmtop
Resolving ambermd.org... 165.230.17.132
Connecting to ambermd.org|165.230.17.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9388148 (9.0M) [text/plain]
Saving to: 'DOPC_128.prmtop.1'

DOPC_128.prmtop.1   100%[===================>]   8.95M  15.9MB/s    in 0.6s    

2016-06-13 13:32:52 (15.9 MB/s) - 'DOPC_128.prmtop.1' saved [9388148/9388148]



In [12]:
view = nv.show_pytraj(traj)
view.clear_representations()
view

In [13]:
view.clear_representations()
view.add_line('not water and not hydrogen')

In [14]:
traj

pytraj.Trajectory, 1 frames: 
Size: 0.000713 (GB)
<Topology: 31882 atoms, 5130 residues, 4874 mols, PBC with box type = ortho>
           

In [15]:
from time import time
xyz = traj[0].xyz.astype('f4')

# Send bytes

In [16]:
mytime = time()*1000

binary = xyz.tobytes()

data_meta = {'0': dict(data=0, dtype='f4', shape=xyz.shape)}

d = dict(type='binary_single', mytime=mytime, data=data_meta)
view.send(d, buffers=[binary])

# Send base64

In [17]:
mytime = time()*1000

data_base64 = {'0': dict(data=nv.encode_base64(xyz, dtype='f4'), 
                  dtype='f4',
                  shape=xyz.shape)}

d = dict(type='base64_single', mytime2=mytime, data=data_base64)

view.send(d)

# make sure we send the same array

In [18]:
from pytraj.testing import aa_eq

xyz_from_bytes = np.frombuffer(binary, dtype='f4').reshape(xyz.shape)
xyz_from_base64 = nv.decode_base64(data_base64['0']['data'], shape=xyz.shape,
                                  dtype='f4')

aa_eq(xyz_from_bytes, xyz_from_base64)